# ToDo
- Look into acceleration data for selecting unusable areas
- Look into RPeaks extraction and PAT

In [1]:
import pytz
import numpy as np
import json
import os
import pandas as pd
import math
import re
import sys
import datetime

import pyedflib
import scipy.signal as sig
from scipy.fftpack import fft
import scipy.stats as stats
import matplotlib.pyplot as plt
import biosppy

import import_ipynb
import glob

ModuleNotFoundError: No module named 'pyedflib'

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import utils.preprocessing_utils as utils

In [ ]:
# Get Experiment Base_Path|
with open('../../config.json') as f:
    paths = json.load(f)
exp_base_path = paths['experiments']
print(exp_base_path)

In [ ]:
exp_date = '2019-02-01/Data'
exp_patient = '002'

patient_base_path = os.path.join(exp_base_path, exp_date, exp_patient)

faros_files = {
    'EDF': '.EDF',
    'ASC': '.ASC',
    'SDF': '.SDF'
}
if os.path.exists(os.path.join(patient_base_path, 'Faros_001')):
    for file in os.listdir(os.path.join(patient_base_path, 'Faros_001')):
        if file.endswith(".EDF"):
            faros_files['EDF'] = file
        elif file.endswith(".ASC"):
            faros_files['ASC'] = file
        elif file.endswith(".SDF"):
            faros_files['SDF'] = file

filelist = {
    'faros_edf': os.path.join(patient_base_path, 'Faros_001', faros_files['EDF']),
    'faros_asc': os.path.join(patient_base_path, 'Faros_001', faros_files['ASC']),
    'faros_sdf': os.path.join(patient_base_path, 'Faros_001', faros_files['SDF']),
    'empatica_acc': glob.glob(os.path.join(patient_base_path, 'Empatica_001/*ACC.csv'))[0],
    'empatica_bvp': glob.glob(os.path.join(patient_base_path, 'Empatica_001/*BVP.csv'))[0],
    #'empatica_eda': glob.glob(os.path.join(patient_base_path, '001/*EDA.csv')),
    #'empatica_hr': glob.glob(os.path.join(patient_base_path, '001/*HR.csv')),
    #'empatica_ibi': glob.glob(os.path.join(patient_base_path, '001/*IBI.csv')),
    #'empatica_temp': glob.glob(os.path.join(patient_base_path, '001/*TEMP.csv')),
    #'everion_signals': os.path.join(patient_base_path, 'Everion/CsvData_signals_EV-EC39-7079-57A8.csv'),
    #'everion_sensors': os.path.join(patient_base_path, 'Everion/CsvData_sensor_data_EV-EC39-7079-57A8.csv'),
    #'everion_aggregates': os.path.join(patient_base_path, 'Everion/CsvData_aggregates_EV-EC39-7079-57A8.csv'),
    #'everion_analytics': os.path.join(patient_base_path, 'Everion/CsvData_analytics_events_EV-EC39-7079-57A8.csv'),
    #'everion_events': os.path.join(patient_base_path, 'Everion/CsvData_everion_events_EV-EC39-7079-57A8.csv'),
    #'everion_features': os.path.join(patient_base_path, 'Everion/CsvData_features_EV-EC39-7079-57A8.csv'),
    'blood_pressure': glob.glob(os.path.join(patient_base_path, 'OMRON_001/*Processed.csv'))[0]
}

## Read Blood Pressure Data

In [ ]:
def load_bp_data(window_size, verbose=False):
    df = pd.read_csv(filelist['blood_pressure'], sep = ",", parse_dates=[0])
    df.rename(columns=lambda x: x.strip().replace("\t",""), inplace = True)
    #df['Sys(mmHg)'] = df['Sys(mmHg)'].str.replace("\t","")
    df['Body Movement'] = df['BodyMovement'].str.replace("\t","")
    df['Step'] = df['Step'].str.replace("\t","")
    bp_df = df.where(df["Step"] == 'BP Measurement').dropna()
    #pd.to_datetime(bp_df["Date and Time of Measurement"][-4:])
    #pd.to_datetime(bp_df["Date and Time of Measurement"].str[-8:])
    bp_df['Measurement Date'] = bp_df["DateandTimeofMeasurement"].str[-8:]
    bp_df['Measurement Date'] = '02/01/2019 ' + bp_df['Measurement Date'].astype(str)
    bp_df['Measurement Date'] = pd.to_datetime(bp_df['Measurement Date'])
    bp_df['Measurement Date'] = bp_df['Measurement Date'] - pd.Timedelta('1 hour')
    bp_df['Window End'] = bp_df['Measurement Date'] + window_size // 2
    bp_df['Window Start'] = bp_df['Measurement Date'] - window_size // 2
    if verbose:
        bp_df.head(50)
        
    return bp_df

## Read PPG Data from Empatica

In [ ]:
def load_empatica_df(bp_df, verbose=False):
    signal_names = ['acc', 'bvp']
    signals = {}
    start_times = {}
    sample_freqs = {}

    # Check start dates
    for signal in signal_names:
        filename = filelist['empatica_{}'.format(signal)]
        with open(filename, 'r') as f:
            ts_line = f.readline()
            start_times[signal] = int(ts_line[:ts_line.find('.')])
            sf_line = f.readline()
            sample_freqs[signal] = int(sf_line[:sf_line.find('.')])
            split = re.split('[,\n]', f.read())
            split = np.array(split[:-1]).astype(float)
            if signal == 'acc':
                split = np.reshape(split, [-1, 3])
            signals[signal] = split

    signals['acc_mag'] = np.linalg.norm(signals['acc'], axis=1)
    del signals['acc']

    sample_freqs['acc_mag'] = sample_freqs['acc']
    del sample_freqs['acc']

    start_times['acc_mag'] = start_times['acc']
    del start_times['acc']

    signal_names.remove('acc')
    signal_names.append('acc_mag')

    data_empatica = utils.create_df(signal_names, signals, sample_freqs, start_times)

    sub_data_empatica = pd.DataFrame()
    for _, row in bp_df.iterrows():
        truncated_df = data_empatica.truncate(before=row['Window Start'], after=row['Window End'])
        if verbose:
            utils.plot_signal(truncated_df.interpolate(method='index'), 'bvp', 'acc_mag')
        sub_data_empatica = sub_data_empatica.append(truncated_df)
    if verbose:
        print(sub_data_empatica.head())
        print(sub_data_empatica.describe())
        utils.plot_signal(sub_data_empatica.interpolate(method='index'), 'bvp', 'acc_mag')
    return sub_data_empatica

In [ ]:
def load_faros_df(bp_df, verbose=False):
    with pyedflib.EdfReader(filelist['faros_edf']) as f:
        # read EDF file into a dictionary, keys: signal_labels 
        n = f.signals_in_file    # get num of signal types 
        signal_labels = f.getSignalLabels()   # get labels for types of signals
        n_samples = f.getNSamples()
        raw_sample_freqs = f.getSampleFrequencies()
        
        ### Dealing with timezone issues
        tz = pytz.timezone('Europe/Berlin')
        initial_ts = datetime.datetime.fromtimestamp(0, tz)
        signal_ts = f.getStartdatetime().replace(tzinfo=pytz.UTC)
        ###
        
        start_ts = (signal_ts - initial_ts).total_seconds()
        print(start_ts)
        raw_data = dict.fromkeys(signal_labels)   # the signals has difference sizes, therefore put into a dictionary
        for i in np.arange(n):
            raw_data[signal_labels[i]] = f.readSignal(i)

    raw_data['acc_mag'] = np.linalg.norm([raw_data['Accelerometer_X'], raw_data['Accelerometer_Y'], raw_data['Accelerometer_Z']], axis=0)
    signal_names = ['ECG', 'acc_mag', 'HRV']
    sample_freqs = {}
    for i in range(len(signal_labels)):
        sample_freqs[signal_labels[i]] = raw_sample_freqs[i]
    sample_freqs['acc_mag'] = sample_freqs['Accelerometer_X']
    start_timestamps = {
        'ECG': start_ts,
        'acc_mag': start_ts,
        'HRV': start_ts
    }
    data_faros = utils.create_df(signal_names, raw_data, sample_freqs, start_timestamps)
    
    sub_data_faros = pd.DataFrame()
    for _, row in bp_df.iterrows():
        truncated_df = data_faros.truncate(before=row['Window Start'], after=row['Window End'])
        if verbose:
            utils.plot_signal(truncated_df.interpolate(method='index'), 'ECG', 'acc_mag')
        sub_data_faros = sub_data_faros.append(truncated_df)
    if verbose:
        print(data_faros.head())
        print(data_faros.describe())
        
    return sub_data_faros

## Utils

In [ ]:
# plots dft of a signal
def plot_dft(signal, freq):
    N = len(signal)
    T = 1/freq
    yf = fft(signal)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    plt.figure(figsize=(14, 6))
    plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
    plt.grid()
    plt.show()

#### Empatica

In [ ]:
# finds the local minima that correspond to the starts of a cardiac cycle
def find_cycle_starts(df, sample_rate=64):
    minima = sig.find_peaks(-df.values, distance=0.7*sample_rate)[0] # Todo: Check other parameters
    #diffs = np.diff(minima)
    #minima = np.delete(minima, np.argwhere(diffs < 0.75*np.mean(diffs)))
    return minima

# returns the x values for those samples in the signal, that are closest to some given y value
def find_xs_for_y(ys, y_val, sys_peak):
    diffs = abs(ys - y_val)
    x1 = diffs[:sys_peak].idxmin()
    x2 = diffs[sys_peak:].idxmin()
    return x1, x2

# takes a dataframe of calculated features and removes the outliers occuding due to inaccuracies in the signal
def clean_window_features_of_outliers(df):
    quant = df.quantile(0.8)
    for col in df.columns:
        if col.find('ts') == -1:
            df = df[df[col] < quant[col]*2]
    return df

# finds sections with high acceleration magnitude and removes them
def remove_motion_sections(df, limit=100, min_size=5, padding=15, std_mult=0.25):
    # Todo check frequency domain for high frequencies
    acc_mag_mean = df['acc_mag'].mean()
    acc_mag_std =  df['acc_mag'].std()
    # Comparison with overall mean and std
    thresh_indices = np.squeeze(np.argwhere((df['acc_mag'] > acc_mag_mean + std_mult * acc_mag_std) | 
                                            (df['acc_mag'] < acc_mag_mean - std_mult * acc_mag_std)))
        
    section_indices = []
    section_start = thresh_indices[0]
    for i in range(1, len(thresh_indices) - 1):
        if thresh_indices[i] - thresh_indices[i-1] > limit:
            if thresh_indices[i-1] >= section_start + min_size:
                section_indices.append((section_start - padding, thresh_indices[i-1] + padding))
            section_start = thresh_indices[i]
    if thresh_indices[-1] != section_start:
        section_indices.append((section_start, thresh_indices[-1]))
    
    # Check local variance for window
    #section_indices = []
    #window_vars = []
    #print(df.count()['bvp'])
    #for i in range((df.count()['bvp'] - limit) // padding + 1):
    #    window = df['acc_mag'].iloc[i*padding:i*padding+limit]
    #    var = window.var()
    #    window_vars.append(var)
    #ind = np.squeeze(np.argwhere(window_vars > np.float64(1)))
    #print(ind)
    #section_indices = []
    #section_start = ind[0]
    #for i in range(1, len(ind) - 1):
    #    if ind[i] - ind[i-1] > 1:
    #        section_indices.append((section_start*padding, ind[i-1]*padding+limit))
    #        section_start = ind[i]
    #if ind[-1] != section_start:
    #    section_indices.append((section_start*padding, ind[-1]*padding+limit))
    #print(section_indices)
        
    section_indices.reverse()
    for (start, end) in section_indices:
        df = df.drop(index=df.iloc[start:end].index)
    return df

def find_clean_cycles_with_template(signal, verbose=False):
    initial_cycle_starts = find_cycle_starts(signal)
    if len(initial_cycle_starts) <= 1:
        return []
    template_length = math.floor(np.median(np.diff(initial_cycle_starts)))
    cycle_starts = initial_cycle_starts[:-1]
    while cycle_starts[-1] + template_length > len(signal):
        cycle_starts = cycle_starts[:-1]
    template = []
    for i in range(template_length):
        template.append(np.mean(signal[cycle_starts + i]))
    
    corr_coef = []
    for cycle_start in cycle_starts:
        corr_coef.append(np.corrcoef(template, signal[cycle_start:cycle_start+template_length])[0,1])

    valid_indices = np.argwhere(np.array(corr_coef) >= 0.8)
    if (len(valid_indices) > len(cycle_starts) / 2) and len(valid_indices) > 1:
        cycle_starts = cycle_starts[np.squeeze(valid_indices)]
        template2 = []
        for i in range(template_length):
            template2.append(np.mean(signal[cycle_starts + i]))
        template = template2
        
    if verbose:
        print('Cycle Template')
        plt.plot(template)
        plt.show()
    
    
    # Check correlation of cycles with template
    # SQI1: Pearson Correlation
    sqi1_corr = []
    for cycle_start in cycle_starts:
        corr, _ = stats.pearsonr(template, signal[cycle_start:cycle_start+template_length])
        sqi1_corr.append(corr)
        
    # SQI2: Pearson Correlation with 
    sqi2_corr = []
    for cycle_start in cycle_starts:
        cycle_end = initial_cycle_starts[np.squeeze(np.argwhere(initial_cycle_starts==cycle_start)) + 1] 
        corr, _ = stats.pearsonr(template, sig.resample(signal[cycle_start:cycle_end], template_length))
        sqi2_corr.append(corr)
        
    # SQI3: Pearson Correlation
    #sq2_corr = []
    #for cycle_start in cycle_starts:
    #    cycle_end = initial_cycle_starts[initial_cycle_starts.index(cycle_start) + 1] 
    #    corr, _ = stats.pearsonr(template, df['bvp'].iloc[cycle_start:cycle_end])
    #    sq2_corr.append(corr)
    corrs = np.array([sqi1_corr, sqi2_corr]).transpose()
    cycle_starts = cycle_starts[np.all(corrs >= 0.8, axis=1)]
    
    if verbose:
        print('Detected Valid Cycles')
        plt.figure()
        for cycle_start in cycle_starts:
            plt.plot(signal[cycle_start:cycle_start+template_length].to_numpy())
        plt.show()
        
    cycles = []
    for cycle_start in cycle_starts:
        cycle_end = initial_cycle_starts[np.squeeze(np.argwhere(initial_cycle_starts==cycle_start)) + 1]
        if (cycle_end - cycle_start) > template_length*1.2:
            cycle_end = cycle_start + template_length
        cycles.append((cycle_start, cycle_end))

    return cycles

#### Faros

In [ ]:
def extract_r_peaks(ecg_signal):
    rpeaks = biosppy.signals.ecg.christov_segmenter(ecg_signal, 1000)
    return rpeaks[0]

def add_rpeaks(data_faros, rpeaks): 
    data_faros['rpeaks'] = 0
    data_faros.iloc[rpeaks, data_faros.columns.get_loc('rpeaks')] = 1

# Filter PPG/ECG Data

In [ ]:
def extract_features_for_cycle(window_df, signal, verbose=False):
    cur_index = window_df.index.max() + 1
    if np.isnan(cur_index):
        cur_index = 0
    signal = signal.resample('ms').nearest(limit=1).interpolate(method='time')
    signal = signal - signal.min()
    max_amplitude = signal.max()
    
    peaks = sig.find_peaks(signal.values)[0]
    sys_peak_ts = signal.index[peaks[0]]
    
    if verbose:
        plt.figure()
        plt.xlim((signal.index.min(), signal.index.max()))
        plt.scatter(signal.index[peaks], signal[peaks])
        plt.plot(signal.index, signal.values)
    # Features
    window_df = window_df.append(pd.DataFrame({'start_ts': signal.index.min(),
                                               'sys_peak_ts': sys_peak_ts,
                                               'T_S': (sys_peak_ts - signal.index.min()).total_seconds(),
                                               'T_D': (signal.index.max() - sys_peak_ts).total_seconds()
                                              }, index=[cur_index]), sort=False)
    for p in [10, 25, 33, 50, 66, 75]:
        p_ampl = p / 100 * max_amplitude
        x1, x2 = find_xs_for_y(signal, p_ampl, peaks[0])
        if verbose:
            plt.scatter([x1, x2], signal[[x1, x2]])
        window_df.loc[cur_index, 'DW_'+str(p)] = (x2 - sys_peak_ts).total_seconds()
        window_df.loc[cur_index, 'DW_SW_sum_'+str(p)] = (x2 - x1).total_seconds()
        window_df.loc[cur_index, 'DW_SW_ratio_'+str(p)] = (x2 - sys_peak_ts) / (sys_peak_ts - x1)
    if verbose:
        plt.show()
    return window_df
    
def extract_features_for_window(df, verbose=False):
    cycles = find_clean_cycles_with_template(df['bvp_normalised'], verbose=verbose)
    if len(cycles) == 0:
        return pd.DataFrame()
    #plt.figure(figsize=(16, 6))
    #plt.plot(df['bvp'].to_numpy())
    #plt.scatter(cycle_starts, df['bvp'].iloc[cycle_starts])
    #plt.show()
    
    window_features = pd.DataFrame()
    cur_index = 0
    for i in range(len(cycles)):
        window_features = extract_features_for_cycle(window_features, df['bvp_normalised'].iloc[cycles[i][0]:cycles[i][1]], verbose=verbose)
        if i > 0:
            window_features.loc[cur_index-1, 'CP'] = (window_features.loc[cur_index, 'sys_peak_ts'] - window_features.loc[cur_index-1, 'sys_peak_ts']).total_seconds()
        cur_index = cur_index + 1
    if verbose:
        print('Cycle Features within Window:')
        print(window_features)
    window_features = clean_window_features_of_outliers(window_features)
    return window_features

def apply_filters(df):
    # No smoothing neccessary due to relatively low sampling frequency
    df['bvp_normalised'] = (df['bvp'] - df['bvp'].min()) / (df['bvp'].max() - df['bvp'].min())
    #butter_f_b, butter_f_a = sig.butter(N=3, Wn=[0.5, 5], btype='bandpass', fs=64)
    #df['bvp_filtered'] = sig.filtfilt(butter_f_b, butter_f_a, df['bvp_normalised'])
    #plt.figure(figsize=(14, 6))
    #plt.plot(df.index, df['bvp_normalised'])
    return df

def extract_features_for_signal(signal, bp, verbose=False):
    for index, row in bp.iterrows():
        window_df = signal.truncate(before=row['Window Start'], after=row['Window End'])
        window_features = extract_features_for_window(window_df, verbose)
        for col in window_features.columns:
            if col.find('ts') == -1:
                bp.loc[index, col+'_mean'] = window_features[col].mean()
                bp.loc[index, col+'_var'] = window_features[col].var()
    bp.dropna(inplace=True)
    return bp

In [ ]:
def extract_PAT_for_window(ppg_df, ecg_df, verbose=False):
    #plt.plot(ppg_df['bvp_normalised'])
    cycles = find_clean_cycles_with_template(ppg_df['bvp_normalised'], verbose=verbose)
    ppg_peak_tss = []
    for i in range(len(cycles)):
        signal = ppg_df['bvp_normalised'].iloc[cycles[i][0]:cycles[i][1]].resample('ms').nearest(limit=1).interpolate(method='time')
        signal = signal - signal.min()
        peaks = sig.find_peaks(signal.values)[0]
        ppg_peak_tss.append(signal.index[peaks[0]])
    PAT_times = []
    # Check if there is an R-Peak before the first detected PPG peak 
    if len(ppg_peak_tss) > 0 and (ppg_peak_tss[0] < ecg_df.index[ecg_df['rpeaks'] == 1]).all():
        del ppg_peak_tss[0]
    if len(ppg_peak_tss) > 0:
        ecg_peak_tss = []
        for ppg_peak_ts in ppg_peak_tss:
            ecg_peak_indices = ecg_df.index[(ecg_df['rpeaks'] == 1) & (ecg_df.index < ppg_peak_ts)]
            if len(ecg_peak_indices) > 0:
                ecg_peak_tss.append(ecg_peak_indices[-1])
        PAT_timedeltas = np.array(ppg_peak_tss) - np.array(ecg_peak_tss)
        for td in PAT_timedeltas:
            PAT_times.append(td.total_seconds())
        if verbose:
            fig, ax1 = plt.subplots(figsize=(15, 6))
            ax2 = ax1.twinx()
            ax2.plot(ppg_df['bvp_normalised'])
            ax1.plot(ecg_df['ECG'], color='r')
            ax2.scatter(ppg_peak_tss, [0.8]*len(ppg_peak_tss))
            ax2.scatter(ecg_peak_tss, [0.8]*len(ecg_peak_tss), color='r')
            fig.tight_layout()
            plt.show()
            print(PAT_times)
    return PAT_times
    
def extract_PAT(ppg_signal, ecg_signal, bp, verbose=False):
    for index, row in bp.iterrows():
        window_ppg = ppg_signal.truncate(before=row['Window Start'], after=row['Window End'])
        window_ecg = ecg_signal.truncate(before=row['Window Start'], after=row['Window End'])
        window_PATs = extract_PAT_for_window(window_ppg, window_ecg, verbose)
        bp.loc[index, 'PAT_mean'] = np.mean(window_PATs)
        bp.loc[index, 'PAT_var'] = np.var(window_PATs)
    bp.dropna(inplace=True)
    return bp

# Main Cells

In [ ]:
bp_df = load_bp_data(pd.Timedelta('1 min'))
sub_data_empatica = load_empatica_df(bp_df)
#utils.plot_signal(sub_data_empatica, 'bvp')
sub_data_empatica = remove_motion_sections(sub_data_empatica)
sub_data_empatica = apply_filters(sub_data_empatica)
#utils.plot_signal(sub_data_empatica, 'bvp')
#utils.plot_signal(sub_data_empatica, 'bvp_normalised')

features = extract_features_for_signal(sub_data_empatica, bp_df, verbose=True)
features.to_csv('../../data/processed/features_{}.csv'.format(exp_patient), index=False)
print(features)

In [ ]:
bp_df = load_bp_data(pd.Timedelta('10 sec'))
sub_data_faros = load_faros_df(bp_df)
rpeaks = extract_r_peaks(sub_data_faros['ECG'].values)
add_rpeaks(sub_data_faros, rpeaks.tolist())
sub_data_empatica = load_empatica_df(bp_df)
#utils.plot_signal(sub_data_empatica, 'bvp')
sub_data_empatica = remove_motion_sections(sub_data_empatica)
sub_data_empatica = apply_filters(sub_data_empatica)

features = extract_PAT(sub_data_empatica, sub_data_faros, bp_df, verbose=True)
features.to_csv('../../data/processed/pat_features_{}.csv'.format(exp_patient), index=False)
print(features)

In [ ]:
!pip install wfdb

In [ ]:
import wfdb
# record_name number_of_signals sampling_frequency number_of_samples_per_signal
# file_name format initial_value checksum block_size description
records = wfdb.io.rdrecord('../../../../Downloads/ecg_data')
print(records.p_signal.shape)
records.p_signal = records.p_signal[100000:101000]
wfdb.plot.plot_wfdb(records, figsize=(16, 16))

In [ ]:
records2 = wfdb.io.rdrecord('../../../../Downloads/s0010_re')
print(records2.p_signal.shape)
print(vars(records2))
wfdb.plot.plot_wfdb(records2, figsize=(16, 16))

In [ ]:
DATA_LOAD_TYPES = {'8': '<i1', '16': '<i2', '24': '<i3', '32': '<i4',
                   '61': '>i2', '80': '<u1', '160': '<u2', '212': '<u1',
                   '310': '<u1', '311': '<u1'}
fmt = '16'
n_samp = 2500
with open('../../../../Downloads/CU025130.821', 'rb') as fp:
    fp.seek(0, 2)
    print(fp.tell())
    fp.seek(1000000)
    sig_data = np.fromfile(fp, dtype=np.dtype(DATA_LOAD_TYPES[fmt]), count=n_samp)
plt.figure(figsize=(16, 6))
plt.plot(sig_data)